In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-01-25 21:12:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.64MB/s    in 0.2s    

2022-01-25 21:12:25 (5.64 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AutomotiveReviewsETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
auto_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
auto_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [38]:
# Drop null values, duplicates, and count number of rows.
auto_df = auto_df.dropna()
auto_df = auto_df.dropDuplicates()
auto_df.count()

3514045

In [39]:
# Show schema to see if it matches the sql schema tables.
auto_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [40]:
# Create dataframe for review_id_table schema
review_id_df = auto_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100B3Y924RGIH|   14305020|B00B0GCZ3I|     881172530| 2014-05-29|
|R100BI6X9TR7G8|   11530488|B00CC2ZU4K|     954296244| 2015-01-21|
|R100IKQF3ORNJF|    9900991|B0091RCKPG|     240957722| 2014-01-10|
|R100PNUOGPNLHB|   22085873|B00EC4OUT2|     726003134| 2015-08-26|
|R100QSKTYYHYBD|   21945587|B001PF4ZZ0|     155818034| 2015-07-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [41]:
# Convert review_date from string to date, check schema.
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType
review_id_df = review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))
review_id_df.printSchema()
  

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [42]:
# Check df to ensure it is in yyyy-mm-dd
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100B3Y924RGIH|   14305020|B00B0GCZ3I|     881172530| 2014-05-29|
|R100BI6X9TR7G8|   11530488|B00CC2ZU4K|     954296244| 2015-01-21|
|R100IKQF3ORNJF|    9900991|B0091RCKPG|     240957722| 2014-01-10|
|R100PNUOGPNLHB|   22085873|B00EC4OUT2|     726003134| 2015-08-26|
|R100QSKTYYHYBD|   21945587|B001PF4ZZ0|     155818034| 2015-07-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [43]:
# Create products_df
products_df = auto_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00B0GCZ3I|Seeutek PK80 80cc...|
|B00CC2ZU4K|OPT7 Bolt AC 55w ...|
|B0091RCKPG|Peerless 0231705 ...|
|B00EC4OUT2|Andoer 2x White U...|
|B001PF4ZZ0|Dynatek Ignition ...|
+----------+--------------------+
only showing top 5 rows



In [44]:
# Create customers_df, need to create customer count column
customers_df = auto_df.select(["customer_id"])
customers_df = customers_df.groupBy("customer_id").count()
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



In [45]:
# Cast count as integer and rename to customer_count
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [46]:
# Preview customers_df
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   38403718|             7|
|   32746205|             1|
|   18898952|             3|
|   14985245|             1|
|   28988871|             3|
+-----------+--------------+
only showing top 5 rows



In [47]:
# Create vine_df
vine_df = auto_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100B3Y924RGIH|          1|            8|         10|   N|
|R100BI6X9TR7G8|          5|            0|          1|   N|
|R100IKQF3ORNJF|          5|            0|          2|   N|
|R100PNUOGPNLHB|          4|            0|          0|   N|
|R100QSKTYYHYBD|          5|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [48]:
# Check vine_df schema
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

